In [ ]:
import random
from copy import deepcopy
from collections import deque
import heapq
from itertools import count  # Import count for unique tie-breaker

class Game:
    def __init__(self):
        # Initialize empty game board
        self.board = [[' ' for _ in range(3)] for _ in range(3)]
        self.current_winner = None  # Keep track of the winner

    def make_move(self, x, y, player):
        # Make a move on the board
        if self.board[x][y] == ' ':
            self.board[x][y] = player
            if self.check_winner(x, y, player):
                self.current_winner = player
            return True
        return False

    def check_winner(self, x, y, player):
        # Check for a winner after each move
        row_win = all([self.board[x][i] == player for i in range(3)])
        col_win = all([self.board[i][y] == player for i in range(3)])
        diag_win = all([self.board[i][i] == player for i in range(3)]) if x == y else False
        anti_diag_win = all([self.board[i][2 - i] == player for i in range(3)]) if x + y == 2 else False
        return row_win or col_win or diag_win or anti_diag_win

    def is_draw(self):
        # Check if the game is a draw
        return all([cell != ' ' for row in self.board for cell in row])

    def available_moves(self):
        # Return list of available moves
        moves = []
        for x in range(3):
            for y in range(3):
                if self.board[x][y] == ' ':
                    moves.append((x, y))
        return moves

    def empty_squares(self):
        # Check for empty squares
        return ' ' in [cell for row in self.board for cell in row]

    def num_empty_squares(self):
        # Number of empty squares
        return len(self.available_moves())

    def print_board(self):
        # Print current state of the board
        for row in self.board:
            print('| ' + ' | '.join(row) + ' |')
        print()

class Player:
    def __init__(self, letter):
        # Player uses 'X' or 'O'
        self.letter = letter

    def get_move(self, game):
        pass

class RandomPlayer(Player):
    def get_move(self, game):
        # Randomly choose a move from available moves
        move = random.choice(game.available_moves())
        return move

class BFSPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)
        self.first_move = True  # Flag to indicate first move

    def get_move(self, game):
        if self.first_move:
            # First move, place randomly
            self.first_move = False
            move = random.choice(game.available_moves())
            return move
        else:
            # Use BFS to decide the next move
            best_score = None
            best_move = None
            for move in game.available_moves():
                copy_game = deepcopy(game)
                copy_game.make_move(move[0], move[1], self.letter)
                score = self.bfs(copy_game)
                if best_score is None or score > best_score:
                    best_score = score
                    best_move = move
            return best_move

    def bfs(self, game):
        # Breadth-first search of the state space
        queue = deque()
        queue.append((deepcopy(game), self.letter))
        while queue:
            current_game, current_letter = queue.popleft()
            if current_game.current_winner == self.letter:
                return 1  # Winning state
            elif current_game.current_winner is not None:
                return -1  # Losing state
            elif current_game.is_draw():
                return 0  # Draw
            for move in current_game.available_moves():
                next_game = deepcopy(current_game)
                next_game.make_move(move[0], move[1], current_letter)
                next_letter = 'O' if current_letter == 'X' else 'X'
                queue.append((next_game, next_letter))
        return 0  # Draw if no winner is found

class DFSPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)
        self.first_move = True  # Flag to indicate first move

    def get_move(self, game):
        if self.first_move:
            # First move, place randomly
            self.first_move = False
            move = random.choice(game.available_moves())
            return move
        else:
            # Use DFS to decide the next move
            best_score = None
            best_move = None
            for move in game.available_moves():
                copy_game = deepcopy(game)
                copy_game.make_move(move[0], move[1], self.letter)
                score = self.dfs(copy_game, self.letter)
                if best_score is None or score > best_score:
                    best_score = score
                    best_move = move
            return best_move

    def dfs(self, game, current_letter):
        # Depth-first search of the state space
        if game.current_winner == self.letter:
            return 1  # Winning state
        elif game.current_winner is not None:
            return -1  # Losing state
        elif game.is_draw():
            return 0  # Draw

        scores = []
        for move in game.available_moves():
            next_game = deepcopy(game)
            next_game.make_move(move[0], move[1], current_letter)
            next_letter = 'O' if current_letter == 'X' else 'X'
            score = self.dfs(next_game, next_letter)
            scores.append(score)
        if current_letter == self.letter:
            return max(scores)  # Maximize score
        else:
            return min(scores)  # Minimize opponent's score

class AStarPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)
        self.first_move = True  # Flag to indicate first move

    def get_move(self, game):
        if self.first_move:
            # First move, place randomly
            self.first_move = False
            move = random.choice(game.available_moves())
            return move
        else:
            # Use A* to decide the next move
            best_score = None
            best_move = None
            for move in game.available_moves():
                copy_game = deepcopy(game)
                copy_game.make_move(move[0], move[1], self.letter)
                score = self.astar(copy_game)
                if best_score is None or score > best_score:
                    best_score = score
                    best_move = move
            return best_move

    def heuristic(self, game):
        # Simple heuristic: count number of potential winning lines
        def count_wins(player):
            lines = [
                # Rows
                [(0,0), (0,1), (0,2)],
                [(1,0), (1,1), (1,2)],
                [(2,0), (2,1), (2,2)],
                # Columns
                [(0,0), (1,0), (2,0)],
                [(0,1), (1,1), (2,1)],
                [(0,2), (1,2), (2,2)],
                # Diagonals
                [(0,0), (1,1), (2,2)],
                [(0,2), (1,1), (2,0)],
            ]
            count = 0
            for line in lines:
                values = [game.board[x][y] for x, y in line]
                if values.count(player) > 0 and values.count('O' if player == 'X' else 'X') == 0:
                    count += 1
            return count

        my_wins = count_wins(self.letter)
        opponent_wins = count_wins('O' if self.letter == 'X' else 'X')
        return my_wins - opponent_wins

    def astar(self, game):
        # A* search of the state space
        heap = []
        counter = count()  # Unique counter to prevent comparison issues
        heapq.heappush(heap, (-self.heuristic(game), next(counter), game, self.letter))
        visited = set()

        while heap:
            _, _, current_game, current_letter = heapq.heappop(heap)
            game_state = tuple(tuple(row) for row in current_game.board)
            if game_state in visited:
                continue
            visited.add(game_state)

            if current_game.current_winner == self.letter:
                return 1  # Winning state
            elif current_game.current_winner is not None:
                return -1  # Losing state
            elif current_game.is_draw():
                return 0  # Draw

            for move in current_game.available_moves():
                next_game = deepcopy(current_game)
                next_game.make_move(move[0], move[1], current_letter)
                next_letter = 'O' if current_letter == 'X' else 'X'
                heuristic_value = self.heuristic(next_game)
                heapq.heappush(heap, (-heuristic_value, next(counter), next_game, next_letter))

        return 0  # Draw if no winner is found

def play_game(player1, player2, show_board=False):
    game = Game()
    current_player = player1  # 'X' always starts
    player2.first_move = False
    if show_board:
        game.print_board()
    while game.empty_squares():
        move = current_player.get_move(game)
        if game.make_move(move[0], move[1], current_player.letter):
            if show_board:
                print(f"Player {current_player.letter} made a move to position ({move[0]}, {move[1]})")
                game.print_board()
            if game.current_winner:
                if show_board:
                    print(f"Player {current_player.letter} wins!")
                return current_player.letter  # Winner
            current_player = player2 if current_player == player1 else player1
        else:
            continue  # Invalid move, same player continues
    if show_board:
        print("The game ended in a draw!")
    return 'Draw'  # Draw

def play_multiple_games(player1_class, player2_class, num_games):
    wins = {'X': 0, 'O': 0, 'Draw': 0}
    for _ in range(num_games):
        player1 = player1_class('X')
        player2 = player2_class('O')
        result = play_game(player1, player2)
        wins[result] += 1
    print(f"Results after {num_games} games:")
    print(f"{player1_class.__name__} (X) won: {wins['X']} times")
    print(f"{player2_class.__name__} (O) won: {wins['O']} times")
    print(f"Draws: {wins['Draw']} times")

def play_single_game_with_display(player1_class, player2_class):
    player1 = player1_class('X')
    player2 = player2_class('O')
    play_game(player1, player2, show_board=True)

In [ ]:
play_multiple_games(RandomPlayer, RandomPlayer, 1000)

In [ ]:
play_multiple_games(BFSPlayer, RandomPlayer, 100)
play_multiple_games(RandomPlayer, BFSPlayer, 100)

In [ ]:
play_multiple_games(DFSPlayer, RandomPlayer, 100)
# play_multiple_games(RandomPlayer, DFSPlayer, 100)

In [ ]:
play_multiple_games(AStarPlayer, RandomPlayer, 100)
play_multiple_games(RandomPlayer, AStarPlayer, 100)

In [ ]:
# play_multiple_games(BFSPlayer, DFSPlayer, 100)
play_multiple_games(DFSPlayer, BFSPlayer, 100)

In [ ]:
play_multiple_games(BFSPlayer, AStarPlayer, 100)
play_multiple_games(AStarPlayer, BFSPlayer, 100)

In [ ]:
play_multiple_games(DFSPlayer, AStarPlayer, 100)
# play_multiple_games(AStarPlayer, DFSPlayer, 100)

In [ ]:
# Play one game with move-by-move display
print("\nSingle game with move-by-move display:")
play_single_game_with_display(AStarPlayer, RandomPlayer)

In [ ]:
class AStarPlayer2(AStarPlayer):
    def __init__(self, letter):
        super().__init__(letter)
        self.first_move = True  # Flag to indicate first move

    def get_move(self, game):
      """
      AStarPlayer doesn't check if there is posibility of immediate winning
      move. It is corrected in this version.
      """
      if self.first_move:
          # First move, place randomly
          self.first_move = False
          move = random.choice(game.available_moves())
          return move
      else:
          # Check for immediate winning move
          for move in game.available_moves():
              copy_game = deepcopy(game)
              copy_game.make_move(move[0], move[1], self.letter)
              if copy_game.current_winner == self.letter:
                  # Immediate win
                  return move

          # Use A* to decide the next move
          best_score = None
          best_move = None
          for move in game.available_moves():
              copy_game = deepcopy(game)
              copy_game.make_move(move[0], move[1], self.letter)
              score = self.astar(copy_game)
              if best_score is None or score > best_score:
                  best_score = score
                  best_move = move
          return best_move

In [ ]:
play_multiple_games(AStarPlayer2, RandomPlayer, 100)
play_multiple_games(RandomPlayer, AStarPlayer2, 100)
play_multiple_games(BFSPlayer, AStarPlayer2, 100)
play_multiple_games(AStarPlayer2, BFSPlayer, 100)
play_multiple_games(DFSPlayer, AStarPlayer2, 100)
# play_multiple_games(AStarPlayer2, DFSPlayer, 100)

In [ ]:
# Play one game with move-by-move display
print("\nSingle game with move-by-move display:")
play_single_game_with_display(AStarPlayer2, RandomPlayer)

In [ ]:
class AStarPlayer3(AStarPlayer):
    def __init__(self, letter):
        super().__init__(letter)
        self.first_move = True  # Flag to indicate first move

    def get_move(self, game):
        """
        AStarPlayer2 doesn't check if there is posibility of oponnent
        immediate winning move. It is corrected in this version.
        """

        if self.first_move:
            # First move, place randomly
            self.first_move = False
            move = random.choice(game.available_moves())
            return move
        else:
            # Check for immediate winning move
            for move in game.available_moves():
                copy_game = deepcopy(game)
                copy_game.make_move(move[0], move[1], self.letter)
                if copy_game.current_winner == self.letter:
                    # Immediate win
                    return move

            # Check for oponnent winning move
            opponent = 'O' if self.letter == 'X' else 'X'
            for move in game.available_moves():
                copy_game = deepcopy(game)
                copy_game.make_move(move[0], move[1], opponent)
                if copy_game.current_winner == opponent:
                    # Block opponent's immediate win
                    return move

            # Use A* to decide the next move
            best_score = None
            best_move = None
            for move in game.available_moves():
                copy_game = deepcopy(game)
                copy_game.make_move(move[0], move[1], self.letter)
                score = self.astar(copy_game)
                if best_score is None or score > best_score:
                    best_score = score
                    best_move = move
            return best_move


In [ ]:
play_multiple_games(AStarPlayer3, RandomPlayer, 100)
play_multiple_games(RandomPlayer, AStarPlayer3, 100)
play_multiple_games(BFSPlayer, AStarPlayer3, 100)
play_multiple_games(AStarPlayer3, BFSPlayer, 100)
play_multiple_games(DFSPlayer, AStarPlayer3, 100)
# play_multiple_games(AStarPlayer3, DFSPlayer, 100)

In [ ]:
# Play one game with move-by-move display
print("\nSingle game with move-by-move display:")
play_single_game_with_display(AStarPlayer3, RandomPlayer)